In [1]:
import re
import os
import pandas as pd

file_paths, file_names, emotions, audios = [], [], [], []
emotion_map = {'anger': 'angry', 'happiness': 'happy', 'sadness': 'sad', 'fear': 'fear',
              'disgust': 'disgust'}

In [2]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

1.12.1+cu113
0.12.1+cu113
cuda


In [3]:
bundle = torchaudio.pipelines.WAV2VEC2_LARGE
extractor = bundle.get_model()
print(extractor.__class__)
print(bundle.sample_rate)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>
16000


In [4]:
import os
os.listdir('../emotiondata/emotion_data')

['anger',
 'session_entries.csv',
 'fear',
 'Tools and Documentation',
 'disgust',
 'sadness',
 'happiness']

In [5]:
folder_list = ['anger', 'disgust', 'fear', 'happiness', 'sadness']
entries = []
for folder in folder_list:
    cur_file_list = os.listdir(f'../emotiondata/emotion_data/{folder}')
    for i in cur_file_list:
        if i == 's05 (3).wav':
            print("found")
            continue
        entries.append(i)


found


In [6]:
import random
random.shuffle(entries)
session = []
equal_parts = (len(entries)-1)//5 # for equally split the entries into 5 parts
count = 0
main_data_path = '../emotiondata/emotion_data'

In [7]:
### Only Run once
from tqdm import tqdm 
folder_map = {'a':'anger', 'd':'disgust', 'f':'fear', 'h':'happiness', 's':'sadness'}

file_paths = []
file_names = []
emotions = []
# audios = []
# labels = []


for i in tqdm(range(len(entries))):
    entry = entries[i]
    if "wav" not in entry:
        continue
    folder = folder_map[entry[0]]
    file_path = f'../emotiondata/emotion_data/{folder}/{entry}'
    emotion = emotion_map[folder]
    file_paths.append(file_path)
    file_names.append(entry)
    emotions.append(emotion)

    # assign session to it
    part = (count//equal_parts)%6 + 1
    if part == 6:
        part = 5
    session.append(part)
    count += 1


file = pd.DataFrame({'path':file_paths, 'name': file_names, 'emotion': emotions, 'session': session})
dataframe_path = main_data_path + '/session_entries.csv'
file.to_csv(dataframe_path)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 604/604 [00:00<00:00, 429528.59it/s]


In [8]:
os.listdir('../emotiondata/emotion_data')

['anger',
 'session_entries.csv',
 'fear',
 'Tools and Documentation',
 'disgust',
 'sadness',
 'happiness']

# Extract Features using Models

In [9]:
import re
import os
import pandas as pd
main_data_path = '../emotiondata/emotion_data'
dataframe_path = main_data_path + '/session_entries.csv'
file = pd.read_csv(dataframe_path)[['path', 'name', 'emotion', 'session']]
file.head()

,path,name,emotion,session
0,../emotiondata/emotion_data/disgust/d07 (5).wav,d07 (5).wav,disgust,1
1,../emotiondata/emotion_data/happiness/h18 (1).wav,h18 (1).wav,happy,1
2,../emotiondata/emotion_data/disgust/d19 (3).wav,d19 (3).wav,disgust,1
3,../emotiondata/emotion_data/disgust/d04 (2).wav,d04 (2).wav,disgust,1
4,../emotiondata/emotion_data/sadness/s12 (5).wav,s12 (5).wav,sad,1


In [11]:
from tqdm import tqdm
import math
audios = []
ct = 0
for i in tqdm(range(math.floor(len(file['path'])))):
    path = file['path'][i]
    wave, sr = torchaudio.load(path)
    if sr != bundle.sample_rate:
        wave = torchaudio.functional.resample(wave, sr, bundle.sample_rate)
    with torch.inference_mode():
        feature, _ = extractor.extract_features(wave)
    
    feature = [f[0] for f in feature]
    audio = torch.stack(feature)
    audios.append(audio)
    ct += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 604/604 [05:47<00:00,  1.74it/s]


# Load Data

In [51]:
holdout = 5
train = file[file['session'] != holdout]
train_audios = [audios[i] for i in range(len(audios)) if file['session'][i] != holdout]
test = file[file['session'] == holdout]
test_audios = [audios[i] for i in range(len(audios)) if file['session'][i] == holdout]

In [52]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, audios, labels, label_transform):
        super(MyDataSet).__init__()
        self.audios = audios
        self.labels = labels
        self.label_transform = label_transform
        
    def __getitem__(self, idx):
        label = self.label_transform[self.labels[idx]]
        audio = self.audios[idx]
        length = audio.size(1)
        return audio, length, label
    
    def __len__(self):
        return len(self.labels)

In [53]:
def collate_indic(data):
    audios, lengths, labels = zip(*data)
    max_len = max(lengths)
    n_ftrs = audios[0].size(2)
    n_dims = audios[0].size(0)
    features = torch.zeros((len(audios), n_dims, max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = audios[i].size(1), audios[i].size(2)
        features[i] = torch.cat([audios[i], torch.zeros((n_dims, max_len - j, k))], dim=1)

    return features, lengths, labels

In [54]:
categories = ['neutral', 'angry', 'happy', 'sad', 'fear', 'disgust']
cate_dic = {}
for i, cate in enumerate(categories):
    cate_dic[cate] = i
cate_dic

{'neutral': 0, 'angry': 1, 'happy': 2, 'sad': 3, 'fear': 4, 'disgust': 5}

In [55]:
from torch.utils.data import DataLoader

train_dataset = MyDataSet(train_audios, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=16, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

test_dataset = MyDataSet(test_audios, test['emotion'].tolist(), cate_dic)
testloader_args = dict(batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, **testloader_args, 
                             collate_fn=collate_indic)

# 3CNN+LSTM

In [56]:
import torch.nn as nn
import torch.nn.functional as F

class ICASSP3CNN(nn.Module):
    def __init__(self, vocab_size, dims = 12, embed_size=128, hidden_size=512, num_lstm_layers = 2, bidirectional = False, label_size=7):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.aggr = nn.Conv1d(in_channels=dims, out_channels=1, kernel_size=1)
        
        self.embed = nn.Linear(in_features = vocab_size, out_features = embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(3 * embed_size)

        self.lstm = nn.LSTM(input_size = 3 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        n, d, b, t = x.size(0), x.size(1), x.size(2), x.size(3)
        x = torch.flatten(x, start_dim=2)
        input = self.aggr(x)
        input = torch.reshape(input, (n, b, t))
        input = self.embed(input)

        batch_size = input.size(0)
        input = input.transpose(1,2)    # (B,T,H) -> (B,H,T)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=False)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]

        logits = self.linear(h_n)

        return logits


In [57]:
from tqdm import tqdm
from torchsummary import summary
import torch.optim as optim

model = ICASSP3CNN(1024, 24)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 50
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    acc_cnt = 0
    err_cnt = 0
    batch_cnt = 0
    model.train()
    for batch, (x, length, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x, length)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().item()

        #model outputs
        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
        batch_cnt += 1
    
    train_loss = train_loss/batch_cnt
    train_accuracy = acc_cnt/(acc_cnt+err_cnt)
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    print(f"epoch:{epoch+1}, train accu:{train_accuracy:.4f},", f"train loss:{train_loss:.2f}")

  2%|█████▊                                                                                                                                                                                                                                                                                          | 1/50 [00:09<07:51,  9.63s/it]

epoch:1, train accu:0.4396, train loss:1.38


  4%|███████████▌                                                                                                                                                                                                                                                                                    | 2/50 [00:19<07:42,  9.64s/it]

epoch:2, train accu:0.6354, train loss:0.96


  6%|█████████████████▎                                                                                                                                                                                                                                                                              | 3/50 [00:28<07:31,  9.60s/it]

...

epoch:48, train accu:1.0000, train loss:0.00


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 49/50 [07:50<00:09,  9.73s/it]

epoch:49, train accu:1.0000, train loss:0.00


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [08:00<00:00,  9.61s/it]

epoch:50, train accu:1.0000, train loss:0.00


# Model Test

In [60]:
test_loss = 0
acc_cnt = 0
err_cnt = 0
batch_cnt = 0
model.eval()

for x, lengths, y in test_dataloader:

    x = x.to(device)
    y = y.to(device)

    logits = model(x, lengths)
    loss = criterion(logits, y)
    test_loss += loss.cpu().item()

    out_val, out_indices = torch.max(logits, dim=1)
    tar_indices = y

    for i in range(len(out_indices)):
        if out_indices[i] == tar_indices[i]:
            acc_cnt += 1
        else:
            err_cnt += 1
    batch_cnt += 1

test_loss = test_loss/batch_cnt
test_accuracy = acc_cnt/(acc_cnt+err_cnt)
print(f'test accuracy: {test_accuracy}')

test accuracy: 0.8225806451612904


In [61]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

aggr.weight tensor([[[-0.1147],
         [ 0.1879],
         [-0.0512],
         [ 0.2427],
         [-0.0038],
         [-0.1734],
         [-0.0730],
         [-0.1916],
         [-0.0427],
         [-0.1285],
         [-0.0452],
         [-0.0993],
         [-0.1508],
         [-0.0767],
         [ 0.2435],
         [-0.0790],
         [ 0.1657],
         [ 0.1471],
         [ 0.1379],
         [ 0.1488],
         [-0.0263],
         [-0.0581],
         [ 0.0352],
         [ 0.0007]]], device='cuda:0')
aggr.bias tensor([0.0769], device='cuda:0')
embed.weight tensor([[-0.0040, -0.0042, -0.0108,  ..., -0.0627,  0.0202, -0.0113],
        [-0.0039, -0.0204, -0.0024,  ...,  0.0384,  0.0085, -0.0265],
        [-0.0347,  0.0233,  0.0070,  ..., -0.0017,  0.0414,  0.0030],
        ...,
        [ 0.0176,  0.0192,  0.0035,  ...,  0.0200, -0.0043, -0.0257],
        [ 0.0402, -0.0169, -0.0363,  ...,  0.0167,  0.0171,  0.0696],
        [-0.0239,  0.0164, -0.0272,  ..., -0.0093,  0.0338,  0.0271]]

In [ ]:
model_path = main_path + f'/models/wav2veclarge/holdout_{holdout}.pth'

torch.save({'epoch':epochs,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict()},
            model_path)